In [1]:
import numpy as np
ddir = './data/'

In [2]:
with open(f'{ddir}movie-xids.npy', 'rb') as f:
    Xids = np.load(f, allow_pickle = True)

with open(f'{ddir}movie-xmask.npy', 'rb') as f:
    Xmask = np.load(f, allow_pickle = True)

with open(f'{ddir}movie-labels.npy', 'rb') as f:
    labels = np.load(f, allow_pickle = True)

In [3]:
Xids

array([[  101,   138,  1326, ...,     0,     0,     0],
       [  101,   138,  1326, ...,     0,     0,     0],
       [  101,   138,  1326, ...,     0,     0,     0],
       ...,
       [  101,   170, 25247, ...,     0,     0,     0],
       [  101,   170, 25247, ...,     0,     0,     0],
       [  101, 22572, 12148, ...,     0,     0,     0]])

In [4]:
labels

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]])

In [5]:
import tensorflow as tf

In [6]:
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

In [7]:
dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(512,), dtype=tf.int32, name=None), TensorSpec(shape=(512,), dtype=tf.int32, name=None), TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [8]:
def map_func(input_ids, masks, labels):
    return {'input_ids': input_ids,
            'attention_mask': masks}, labels

In [9]:
dataset = dataset.map(map_func)

In [10]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.int32, name=None)}, TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [11]:
batch_size = 16
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder = True)

In [12]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)}, TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))>

In [13]:
split = 0.9
size = int((Xids.shape[0] / batch_size)* split)

In [14]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

In [15]:
tf.data.Dataset.save(train_ds, f'{ddir}train')
tf.data.Dataset.save(val_ds, f'{ddir}val')

In [16]:
ds = tf.data.Dataset.load(f'{ddir}train', element_spec = train_ds.element_spec)

In [17]:
ds.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)}, TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))>